<a href="https://colab.research.google.com/github/abhishekthapa123/AnnimeRecommendationSystem/blob/master/anime_recommendation_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Anime Recommendation project

In [ ]:
!pip install kaggle lightfm

!mkdir /root/.kaggle

!ls /root/.kaggle

open('/root/.kaggle/kaggle.json','w').write('{"username":"#your user name","key":"your key for kaggle"}')
!chmod 600 /root/.kaggle/kaggle.json


    100% |████████████████████████████████| 307kB 12.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [ ]:
!kaggle datasets download CooperUnion/anime-recommendations-database

 48% 12.0M/25.2M [00:00<00:00, 23.6MB/s]
100% 25.2M/25.2M [00:00<00:00, 51.3MB/s]


In [ ]:
!unzip anime-recommendations-database.zip

Archive:  anime-recommendations-database.zip
  inflating: anime.csv               
  inflating: rating.csv              


In [ ]:
!wget https://raw.githubusercontent.com/pytholabsbot1/anime_recommender/master/recsys.py
!wget https://raw.githubusercontent.com/pytholabsbot1/anime_recommender/master/generic_sys.py

--2019-04-05 13:23:41--  https://raw.githubusercontent.com/pytholabsbot1/anime_recommender/master/recsys.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8190 (8.0K) [text/plain]
Saving to: ‘recsys.py’

recsys.py           100%[===================>]   8.00K  --.-KB/s    in 0s      

2019-04-05 13:23:41 (84.2 MB/s) - ‘recsys.py’ saved [8190/8190]

--2019-04-05 13:23:43--  https://raw.githubusercontent.com/pytholabsbot1/anime_recommender/master/generic_sys.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10609 (10K) [text/plain]
Saving to

In [ ]:
# Importing Libraries and cookbooks
from recsys import * ## recommender system cookbook
from generic_sys import * ## pre-processing code
from IPython.display import HTML ## Setting display options for Ipython Notebook
import pandas as pd,numpy as np

In [ ]:
anime = pd.read_csv('anime.csv')
ratings = pd.read_csv('rating.csv').iloc[:100000]

ids = np.unique(ratings.anime_id)

anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [ ]:
anime_dict = {}

for i,el in enumerate(anime.anime_id.values):
  if(el in ids):
    #print(el)
    anime_dict[el] = anime.name.values[i]
    #break
    
print(anime_dict)

{32281: 'Kimi no Na wa.', 5114: 'Fullmetal Alchemist: Brotherhood', 28977: 'Gintama°', 9253: 'Steins;Gate', 9969: 'Gintama&#039;', 32935: 'Haikyuu!!: Karasuno Koukou VS Shiratorizawa Gakuen Koukou', 11061: 'Hunter x Hunter (2011)', 820: 'Ginga Eiyuu Densetsu', 15335: 'Gintama Movie: Kanketsu-hen - Yorozuya yo Eien Nare', 15417: 'Gintama&#039;: Enchousen', 4181: 'Clannad: After Story', 918: 'Gintama', 2904: 'Code Geass: Hangyaku no Lelouch R2', 28891: 'Haikyuu!! Second Season', 199: 'Sen to Chihiro no Kamikakushi', 23273: 'Shigatsu wa Kimi no Uso', 24701: 'Mushishi Zoku Shou 2nd Season', 12355: 'Ookami Kodomo no Ame to Yuki', 1575: 'Code Geass: Hangyaku no Lelouch', 263: 'Hajime no Ippo', 44: 'Rurouni Kenshin: Meiji Kenkaku Romantan - Tsuioku-hen', 1: 'Cowboy Bebop', 30276: 'One Punch Man', 164: 'Mononoke Hime', 7311: 'Suzumiya Haruhi no Shoushitsu', 17074: 'Monogatari Series: Second Season', 21939: 'Mushishi Zoku Shou', 457: 'Mushishi', 2001: 'Tengen Toppa Gurren Lagann', 245: 'Great T

In [ ]:
# Creating interaction matrix using rating data
interactions = create_interaction_matrix(df = ratings,
                                         user_col = 'user_id',
                                         item_col = 'anime_id',
                                         rating_col = 'rating')
interactions.shape

(1028, 5154)

In [ ]:
mf_model = runMF(interactions = interactions,
                 n_components = 30,
                 loss = 'warp',
                 epoch = 30,
                 n_jobs = 4)

In [ ]:
## Creating item-item distance matrix
item_item_dist = create_item_emdedding_distance_matrix(model = mf_model,interactions = interactions)
## Checking item embedding distance matrix
#item_item_dist.head()


In [ ]:
## Calling 10 recommended items for item id 
rec_list = item_item_recommendation(item_emdedding_distance_matrix = item_item_dist,
                                    item_id = 1535,
                                    item_dict = anime_dict,
                                    n_items = 10)

Item of interest :Death Note
Item similar to the above item:
1- Code Geass: Hangyaku no Lelouch
2- Elfen Lied
3- Shingeki no Kyojin
4- Another
5- Soul Eater
6- Durarara!!
7- Fullmetal Alchemist
8- Code Geass: Hangyaku no Lelouch R2
9- Steins;Gate
10- Deadman Wonderland


In [ ]:
rec_list

[501, 5005, 4773, 1157, 9119, 956, 6583, 4375, 636, 6582]

In [ ]:
l = anime.name[ratings.anime_id.unique()]

/usr/local/lib/python3.6/dist-packages/pandas/core/series.py:696: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


In [ ]:
q = input('anime key: ')

for i in anime_dict:
  if(q in anime_dict[i].lower()):
    print(i,anime_dict[i])

anime key: death
1535 Death Note
28223 Death Parade
14353 Death Billiards
2994 Death Note Rewrite
31 Neon Genesis Evangelion: Death &amp; Rebirth
10444 Digimon Xros Wars: Aku no Death General to Nanatsu no Oukoku
